In [1]:
%pylab notebook
# check if a windows machine, it needs special attention
# this extra step will bypass an error from mpl_toolkits.basemap
import os
if os.name == 'nt':
    os.environ["PROJ_LIB"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share")
    os.environ["GDAL_DATA"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share", "gdal")
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from netCDF4 import Dataset, num2date
import numpy as np
from osgeo import gdal, osr   # noqa
import pandas as pd
from pathlib import Path
import re
from cetbtools.ease2conv import Ease2Transform
from mpl_toolkits.basemap import Basemap
import glob
from pathlib import Path

Populating the interactive namespace from numpy and matplotlib


# Read a GRD MOD pkl file and corresponding lat/lons

In [2]:
dataDir = Path(Path.home(), 'nsidc0630_v1') # Mariah's PC or Mary Jo's Mac
scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')

modsDir = "%s/MODs" % dataDir
modsDir

'/Users/mj/nsidc0630_v1/MODs'

In [3]:
os.chdir(modsDir)
modFiles = sort(glob.glob("WesternCA*MOD.pkl"))
modFiles

array(['WesternCA.bathurst_range.F13.37V.GRD.MOD.pkl',
       'WesternCA.bathurst_range.F13.37V.SIR.MOD.pkl'], dtype='<U44')

In [4]:
geoFiles = sort(glob.glob("WesternCA*geolocation.pkl"))
geoFiles

array(['WesternCA.bathurst_range.F13.37V.GRD.geolocation.pkl',
       'WesternCA.bathurst_range.F13.37V.SIR.geolocation.pkl'],
      dtype='<U52')

In [87]:
fidx = 0
MOD_df = pd.read_pickle(modFiles[fidx])
geo_df = pd.read_pickle(geoFiles[fidx])

In [88]:
MOD_df

,1995,1996,1997,Avg
"(row,col)",,,,
"42,7",128.0,108.0,97.0,111.000000
"42,8",129.0,73.0,97.0,99.666667
"42,9",128.0,106.0,105.0,113.000000
"42,10",127.0,73.0,97.0,99.000000
"42,11",127.0,49.0,95.0,90.333333
...,...,...,...,...
"93,35",NaN,143.0,140.0,141.500000
"93,36",283.0,147.0,140.0,190.000000
"93,37",NaN,142.0,NaN,142.000000


In [89]:
geo_df

,"42,7","42,8","42,9","42,10","42,11","42,12","42,13","42,14","42,15","42,16",...,"93,30","93,31","93,32","93,33","93,34","93,35","93,36","93,37","93,38","93,39"
Latitude,54.298965,54.50116,54.702831,54.903973,55.104580,55.304648,55.504171,55.703143,55.901558,56.099410,...,63.943685,64.166094,64.388264,64.610192,64.831876,65.053314,65.274502,65.495438,65.716120,65.936544
Longitude,-120.901070,-121.08982,-121.280655,-121.473605,-121.668703,-121.865978,-122.065463,-122.267190,-122.471192,-122.677503,...,-104.819418,-104.947552,-105.077870,-105.210426,-105.345277,-105.482482,-105.622100,-105.764195,-105.908832,-106.056077


# Merge the MOD df with the geolocation information df

In [71]:
def parse_row_col(s):
    return [int(str) for str in s.split(',')]

In [90]:
def merge_MOD_and_geo_dfs(MOD_df, geo_df, verbose=False):

    # Put lat/lon into new columns
    tmp = geo_df.transpose()
    df = tmp.join(MOD_df)
    if (verbose):
        print("with new lat/lon columns:")
        print(df)
        
    # Move row, column information from the index to their own columns               
    df.reset_index(inplace=True)
    df.rename(columns={'index':'pixel'}, inplace=True)
    if (verbose):
        print("with new pixel column:")
        print(df)
    
    df["Row"] = df["Column"] = ""
    df[["Row", "Column"]] = list(df.pixel.apply(parse_row_col))
    if (verbose):
        print("with new Row/Column columns:")
        print(df)
    
    return df

In [91]:
df = merge_MOD_and_geo_dfs(MOD_df, geo_df)
df

,pixel,Latitude,Longitude,1995,1996,1997,Avg,Row,Column
0,"42,7",54.298965,-120.901070,128.0,108.0,97.0,111.000000,42,7
1,"42,8",54.501160,-121.089820,129.0,73.0,97.0,99.666667,42,8
2,"42,9",54.702831,-121.280655,128.0,106.0,105.0,113.000000,42,9
3,"42,10",54.903973,-121.473605,127.0,73.0,97.0,99.000000,42,10
4,"42,11",55.104580,-121.668703,127.0,49.0,95.0,90.333333,42,11
...,...,...,...,...,...,...,...,...,...
1711,"93,35",65.053314,-105.482482,NaN,143.0,140.0,141.500000,93,35
1712,"93,36",65.274502,-105.622100,283.0,147.0,140.0,190.000000,93,36
1713,"93,37",65.495438,-105.764195,NaN,142.0,NaN,142.000000,93,37
1714,"93,38",65.716120,-105.908832,NaN,142.0,142.0,142.000000,93,38


# Initialize a grid object for the transformations between lat/lon, row/col and x/y

This will depend on whether data are GRD (25 km) or SIR (6.25 or 3.125)

In [45]:
from cetbtools import ease2conv
N25grid = ease2conv.Ease2Transform(gridname="EASE2_N25km")
N3grid = ease2conv.Ease2Transform(gridname="EASE2_N3.125km")

# Function that takes a column of the original MOD df and reshapes it to a 2D array

In [85]:
def write_df_column_to_geotiff(df, column, outfilename, grid, dtype='int16', verbose=False):

    nrows = int(df.iloc[-1].Row - df.iloc[0].Row + 1)
    ncols = int(df.iloc[-1].Column - df.iloc[0].Column + 1)
    
    # Coerce array data to requested dtype
    data = np.array(df[column].values.data).reshape(nrows, ncols)
    data = data.astype(dtype)
    if verbose:
        print('data dtype = %s, nrows = %d, ncols = %d' % 
              (data.dtype, nrows, ncols),
             file=sys.stderr,
             flush=True)
    
    if ("float32" == data.dtype):                                                                    
        gdal_data_type = gdal.GDT_Float32                                                       
    elif ("int8" == data.dtype):                                                                  
        gdal_data_type = gdal.GDT_Byte                                                          
    elif ("int16" == data.dtype):                                                                  
        gdal_data_type = gdal.GDT_Int16                                                         
    else:                                                                                       
        print("%s : Unrecognized type %s " %                                                 
              (my_name, str(data.dtype)),                                             
              file=sys.stderr,                                                                  
              flush=True)                                                                       
        raise ValueError                                                                        

    # Initialize the output driver
    # Documentation for raster GTiff driver here: https://gdal.org/drivers/raster/gtiff.html#raster-gtiff
    driver = gdal.GetDriverByName("GTiff")                                                      
                                                                                                
    # use this to control block sizes:
    # dest_ds_options = ['COMPRESS=LZW', 'TILED=YES', 'BLOCKXSIZE=256', 'BLOCKYSIZE=256']  
    dest_ds_options = ['COMPRESS=LZW']
    dest_ds = driver.Create(outfilename, ncols, nrows, 1, gdal_data_type, dest_ds_options)
    
    # Initialize the projection information                                                     
    # The crs.proj4text attribute can also be used here,                                        
    # but the srid attribute provides more complete PROJCRS metadata                            
    proj = osr.SpatialReference()                                                                                                                   
    proj.SetFromUserInput(grid.epsg)                                                             
    dest_ds.SetProjection(proj.ExportToWkt())
    
    # Initialize the grid information (extent and scale)                                        
    # Thanks to web page at:                                                                    
    # http://geoexamples.blogspot.com/2012/01/                                                  
    # creating-files-in-ogr-and-gdal-with.html                                                  
    # The geotransform defines the relation between the                                         
    # raster coordinates x, y and the                                                           
    # geographic coordinates, using the following definition:                                   
    # Xgeo = geotransform[0] + Xpixel*geotransform[1] + Yline*geotransform[2]                   
    # Ygeo = geotransform[3] + Xpixel*geotransform[4] + Yline*geotransform[5]                   
    # The first and fourth parameters define the origin of the upper left pixel                 
    # The second and sixth parameters define the pixels size.                                   
    # The third and fifth parameters define the rotation of the raster.                         
    # Values are meters                                                                         
    # The UL information is the center of the UL corner pixel in projected
    # coordinates
    ULrow, ULcol = grid.geographic_to_grid(df.iloc[0].Latitude, df.iloc[0].Longitude)
    print(ULrow, ULcol)
    ULrow = int(ULrow + 0.5)
    ULcol = int(ULcol + 0.5)
    
    map_ULx, map_ULy = grid.grid_to_map(ULrow, ULcol)
    print("UL: ", ULrow, ULcol, df.iloc[0].Latitude, df.iloc[0].Longitude, map_ULx, map_ULy)
    
    LRrow, LRcol = grid.geographic_to_grid(df.iloc[-1].Latitude, df.iloc[-1].Longitude)
    LRrow = int(LRrow + 0.5)
    LRcol = int(LRcol + 0.5)
    map_LRx, map_LRy = grid.grid_to_map(LRrow, LRcol)
    print("LR: ", LRrow, LRcol, df.iloc[-1].Latitude, df.iloc[-1].Longitude, map_LRx, map_LRy)
    
    # Get the projection scales by dividing the projected extents from UL and LR pixels
    # by row/col dimensions
    scale_x = (map_LRx - map_ULx) / double(ncols - 1)
    scale_y = -1. * (map_ULy - map_LRy) / double(nrows - 1)
    print('scale x, y = %f, %f' % (scale_x, scale_y))

    cornerULx = map_ULx - (scale_x / 2.)
    cornerULy = map_ULy - (scale_y / 2.)
    print("cornerUL: ", cornerULx, cornerULy)
    geotransform = (cornerULx, scale_x, 0., cornerULy, 0., scale_y)                               
    dest_ds.SetGeoTransform(geotransform)                                                       
    dest_ds.GetRasterBand(1).WriteArray(data) 
                                                     
    dest_ds = None

    if verbose:                                                                                 
        print("\n%s geotiff image saved to: %s" %                                          
              (str(column), outfilename),                                                 
              file=sys.stderr,                                                                  
              flush=True)  

In [29]:
ULrow, ULcol = N25grid.geographic_to_grid(df.iloc[0].Latitude, df.iloc[0].Longitude)
print(ULrow, ULcol)
ULrow = int(ULrow + 0.5)
ULcol = int(ULcol + 0.5)
print(ULrow, ULcol)
ULx, ULy = N25grid.grid_to_map(ULrow, ULcol)
print(ULx, ULy)

278.9999999952083 224.99999999199392
279 225
-3362500.0 2012500.0


In [93]:
column = 1995
outfilename = 'test.tif'
grid = N25grid
write_df_column_to_geotiff(df, column, outfilename, grid, verbose=True)

data dtype = int16, nrows = 52, ncols = 33

1995 geotiff image saved to: test.tif


278.9999999952083 224.99999999199392
UL:  279 225 54.29896460555291 -120.9010697086037 -3362500.0 2012500.0
LR:  330 257 65.93654385712269 -106.05607659939838 -2562500.0 737500.0
scale x, y = 25000.000000, -25000.000000
cornerUL:  -3375000.0 2025000.0


In [65]:
2012500 + 12500.

2025000.0

In [40]:
%pwd


'/Users/mj/nsidc0630_v1/MODs'

In [ ]:
for column in MOD_df:
    print('next array is %s' % column)
    data = np.array(MOD_df[column].values.data).reshape(nrows, ncols)
    print(vals.dtype)

In [ ]:
data = np.reshape()

# Figure out if (42,7) is relative to EASE-Grid 2.0 or WesternCA subset

In [ ]:
from cetbtools import ease2conv
Ngrid = ease2conv.Ease2Transform(gridname="EASE2_N25km")

In [ ]:
geo_df['42,7']

In [ ]:
geo_df.iloc[:,0].Latitude, geo_df.iloc[:,0].Longitude

In [ ]:
ULrow, ULcol = Ngrid.geographic_to_grid(geo_df.iloc[:,0].Latitude, geo_df.iloc[:,0].Longitude)
ULrow, ULcol

In [ ]:
Ngrid.grid_to_geographic(279, 225)

In [ ]:
#help(Ngrid)

In [ ]:
(x, y) Ngrid.grid_to_map(279, 225)

In [ ]:
np.floor(np.array([-2.5, -1.5, -0.5, 0.0, 0.5, 1.5]) + 0.5)

# Function that takes a column of the original MOD df and reshapes it to a 2D array